# 利用预训练模型ResNet50分类图像
#### 参考资料


In [6]:
import os
import numpy as np
import h5py
import tensorflow as tf
import keras
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
from keras.preprocessing.image import ImageDataGenerator
from mylibs.data.ImageNetData import ImageNetData
from mylibs import funs

#=========参数配置=========
#ImageNet Data配置目录
imageNet_data_path='%s/e/dataset_tiptical/image_net/ILSVRC2012_devkit_t12/data'%os.getenv('HOME') #ImageNetData
#待分类图像目录
images_path='%s/e/dataset_tiptical/flowers17'%os.getenv('HOME')
#软链接输出目录
to_path  ='%s/work/temp/resnet50_flowers17'%os.getenv('HOME')

#========加载模型==========
model=ResNet50(weights='imagenet')
model.summary()

#========加载ImageNetData配置信息=============
dataInfo=ImageNetData(imageNet_data_path)

#========收集分类图像=========================
files=funs.GatherFiles(images_path,exts='.jpg',exclude_files='url')

#========图像分类============================
#ImageNet测试数据集
import os
import h5py
from keras.preprocessing import image
from mylibs.ProcessBar import ShowProcess
pb = ShowProcess(100,'','', 'OK') #进度条

#逐个文件测试
count=len(files)
for i,sfile in enumerate(files):
    src=os.path.join(images_path,sfile)                   #图像完整路径
    if not os.path.isdir(src):
        img = image.load_img(src, target_size=(224, 224)) #加载图像
        x = image.img_to_array(img)    #to_array
        x = np.expand_dims(x, axis=0)  #扩展batch_size维度
        x = preprocess_input(x)        #图像预处理
        preds = model.predict(x)       #模型预测
        #print('Predicted:', decode_predictions(preds, top=3)[0])
        # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]
        #top3结果转换为3x2数组，第一列为预测ID，第二列为置信度;第一行至第三行分别为top1,2,3
        top3=decode_predictions(preds,top=3)[0] #解码top3结果
        wnid=top3[0][0]
        f_info=dataInfo.get_info_from_wnid(wnid)
        if not f_info is None:
            dst_path='%s/%s_%s'%(to_path,f_info['WNID'],f_info['cn_name'])
            dst='%s/%s'%(dst_path,os.path.basename(sfile))
            free_index=1
            while True:                
                if os.path.exists(dst):
                    splits=os.path.splitext(sfile) #=>('/home/temp/file','.txt')
                    dst='%s/%s_%d.%s'%(dst_path,os.path.basename(splits[0]),free_index,splits[1])
                else:
                    break;
            os.makedirs(dst_path) if not os.path.exists(dst_path) else ''
            os.symlink(src,dst)            
        if i % 10 == 0:
            pb.show_process(int(i*100.0/count),infoTail='%d/%d'%(i,count))
pb.show_process(100)

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_6[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

[>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>]1200/1203
OK
